In [577]:
import cv2
import numpy as np
import pandas as pd
import joblib
from sklearn.linear_model import SGDRegressor, Ridge, ElasticNet, Lasso
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import RidgeCV, ElasticNetCV, LassoCV

In [576]:
import skimage
from skimage import io
from skimage import transform

In [578]:
df = pd.read_csv('/content/drive/MyDrive/Data - Sheet1.csv')
df

,Path,Identity,Expression,Age
0,/content/drive/MyDrive/Data /imgNot1.jpeg,Unknown,Not smiling,35
1,/content/drive/MyDrive/Data /imgNot2.jpeg,Unknown,Not smiling,35
2,/content/drive/MyDrive/Data /imgSmile3.jpg,Unknown,Smiling,35
3,/content/drive/MyDrive/Data /imgSmile4.jpeg,Unknown,Smiling,35
4,/content/drive/MyDrive/Data /imgSmile5.jpeg,Unknown,Not smiling,37
5,/content/drive/MyDrive/Data /imgNot6.jpeg,Unknown,Not smiling,34
6,/content/drive/MyDrive/Data /imgNot7.jpg,Unknown,Not smiling,40
7,/content/drive/MyDrive/Data /imgNot9.jpg,Unknown,Not smiling,42
8,/content/drive/MyDrive/Data /imgNot10.jpg,Unknown,Not smiling,47
9,/content/drive/MyDrive/Data /imgNot11.png,Unknown,Not smiling,32


In [579]:
#Part 1
model_sdg = joblib.load('/content/drive/MyDrive/sdg_regressor.pkl')
model_sdg

SGDRegressor(random_state=42)

In [580]:
x = df[['Path', 'Identity', 'Expression']]
y = df['Age']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [581]:
#Specifying the path
path = x_train['Path']
arr = []
for files in path:
    image = io.imread(files)
    #resizing the image to match a certain type
    img_resize = transform.resize(image, output_shape=(32,32,3))
    img_reshape = np.reshape(img_resize, (32*32*3))
    arr.append(img_reshape/255.)

x_train = np.array(arr)
print(x_train)

[[3.09446212e-03 2.47839806e-03 1.72065509e-03 ... 3.85575322e-03
  3.03844367e-03 2.38952832e-03]
 [3.05173704e-03 3.09787314e-03 3.17476664e-03 ... 2.13957711e-04
  2.13949125e-04 2.13962005e-04]
 [3.77392151e-03 3.63632847e-03 3.20809474e-03 ... 1.68404898e-03
  1.41684265e-03 8.42342840e-04]
 ...
 [2.62988737e-04 2.56123265e-04 1.74823844e-04 ... 3.95304653e-05
  4.97805434e-05 6.03759890e-05]
 [3.81842583e-03 3.72056645e-03 3.43871805e-03 ... 7.39322195e-04
  6.79304407e-04 6.69993975e-04]
 [3.71208972e-04 3.76447714e-04 4.79417786e-04 ... 1.85472015e-03
  1.56230761e-03 1.33006455e-03]]


In [582]:
param_grid = {
    'penalty': ['l2', 'l1', 'elasticnet', None],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0]
}

In [583]:
grid_search = GridSearchCV(model_sdg, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=42), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],
                         'penalty': ['l2', 'l1', 'elasticnet', None]},
             scoring='neg_mean_squared_error')

In [584]:
best_penalty = grid_search.best_params_['penalty']
best_alpha_sgd = grid_search.best_params_['alpha']

In [585]:
print("Best Penalty:", best_penalty)
print("Best Alpha (SGD):", best_alpha_sgd)

Best Penalty: l1
Best Alpha (SGD): 0.001


In [586]:
sdg_best_model = SGDRegressor(penalty=best_penalty, alpha=best_alpha_sgd)
sdg_best_model.fit(x_train, y_train)

SGDRegressor(alpha=0.001, penalty='l1')

In [587]:

#Specifying the path
path = x_test['Path']
arr = []
for files in path:
    image = io.imread(files)
    #resizing the image to match a certain type
    img_resize = transform.resize(image, output_shape=(32,32,3))
    img_reshape = np.reshape(img_resize, (32*32*3))
    arr.append(img_reshape/255.)

x_test = np.array(arr)
print(x_test)

[[3.27019099e-03 3.27019099e-03 3.27039496e-03 ... 2.98828558e-03
  2.32099285e-03 2.08039890e-03]
 [4.61361015e-05 6.15148020e-05 1.23029604e-04 ... 4.61361015e-05
  3.07574010e-05 1.07650903e-04]
 [2.43262580e-03 1.49256821e-03 1.10190258e-03 ... 2.99419966e-03
  2.27141316e-03 1.91344944e-03]
 ...
 [1.91335184e-05 1.01598552e-05 1.07777587e-05 ... 1.24979804e-03
  6.63065376e-04 3.84032231e-04]
 [3.91539925e-03 3.91539922e-03 6.52566631e-04 ... 3.91062680e-03
  3.91057407e-03 6.54603535e-04]
 [1.72153503e-03 1.84457426e-03 1.89070555e-03 ... 3.57024866e-03
  2.96039886e-03 2.56318485e-03]]


In [588]:
y_pred_sdg_best = sdg_best_model.predict(x_test)


In [589]:
r2_sdg_best = r2_score(y_test, y_pred_sdg_best)
mse_sdg_best = mean_squared_error(y_test, y_pred_sdg_best)

print("SGDRegressor with Optimal Parameters:")
print(f'R2 Score {r2_sdg_best}')
print(f'Mean Square Error {mse_sdg_best}')

SGDRegressor with Optimal Parameters:
R2 Score -0.0969141404750482
Mean Square Error 67.44308035577055


In [590]:
#Part 3:
alphas = [0.0001, 0.001, 0.01, 0.1, 1]

In [604]:
#Ridge CV
ridge_cv_model = RidgeCV(alphas=alphas, cv=5)
ridge_cv_model.fit(x_train, y_train)
best_alpha_ridge = ridge_cv_model.alpha_
print(f'Optimal Alpha Ridge {best_alpha_ridge}')

Optimal Alpha Ridge 0.01


In [605]:
#Elasticnet CV
elasticnet_cv_model = ElasticNetCV(alphas=alphas, cv=5)
elasticnet_cv_model.fit(x_train, y_train)
best_alpha_elasticnet = elasticnet_cv_model.alpha_
print(f"Optimal Alpha ElasticNet: {best_alpha_elasticnet}")

Optimal Alpha ElasticNet: 0.0001


In [606]:
#Lasso CV
lasso_cv_model = LassoCV(alphas=alphas, cv=5)
lasso_cv_model.fit(x_train, y_train)
best_alpha_lasso = lasso_cv_model.alpha_
print(f"Optimal Alpha Lasso: {best_alpha_lasso}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.023e+00, tolerance: 2.160e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.292e-01, tolerance: 2.366e-01
  model = cd_fast.enet_coordinate_descent(


Optimal Alpha Lasso: 1.0


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.929e-01, tolerance: 2.529e-01
  model = cd_fast.enet_coordinate_descent(


In [607]:
#Part 4
#The ridge elasticnet and lasso are trained with best(optimal) Alphas


In [608]:
#Ridge
ridge_model = Ridge(alpha=best_alpha_ridge)
ridge_model.fit(x_train, y_train)
y_pred_ridge = ridge_model.predict(x_test)

In [609]:
#Elasticnet
elasticnet_model = ElasticNet(alpha=best_alpha_elasticnet)
elasticnet_model.fit(x_train, y_train)
y_pred_elasticnet = elasticnet_model.predict(x_test)

In [610]:
#Lasso
lasso_model = Lasso(alpha=best_alpha_lasso)
lasso_model.fit(x_train, y_train)
y_pred_lasso = lasso_model.predict(x_test)

In [611]:
r2_ridge = r2_score(y_test, y_pred_ridge)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)

r2_elasticnet = r2_score(y_test, y_pred_elasticnet)
mse_elasticnet = mean_squared_error(y_test, y_pred_elasticnet)

r2_lasso = r2_score(y_test, y_pred_lasso)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)

In [612]:
#Evaluating Ridge:

print("Ridge Regression:")
print(f"R2 Score: { r2_ridge}")
print(f"Mean Squared Error: {mse_ridge}")

Ridge Regression:
R2 Score: 0.243737812956616
Mean Squared Error: 46.49830790649556


In [613]:
#Evaluating Elasticnet:

print("ElasticNet Regression:")
print(f"R2 Score: {r2_elasticnet}")
print(f"Mean Squared Error: {mse_elasticnet}")

ElasticNet Regression:
R2 Score: 0.26794362889725354
Mean Squared Error: 45.01002844202043


In [614]:
#Evaluating Lasso

print("Lasso Regression:")
print(f"R2 Score: {r2_lasso}")
print(f"Mean Squared Error: {mse_lasso}")

Lasso Regression:
R2 Score: -0.10420902160101653
Mean Squared Error: 67.8916015625


In [603]:
#When comparing the above models with OLS model Elasticnet model seems the optimal as it has least error
joblib.dump(elasticnet_model, '/content/drive/MyDrive/elasticnet_model.pkl')

['/content/drive/MyDrive/elasticnet_model.pkl']